## distributed demo
this notebook shows how to run distributed julia programs.   it assumes you have a version of julia running on a machine on your local network.   In this case the worker container was run with the command 

<pre>
docker run -it -d -p 3456:22 dbgannon/juliacloud2
</pre>
the IP address of the machine where this was run is known to be 10.0.0.29.

In [1]:
using Distributed
workers()

1-element Array{Int64,1}:
 1

In [2]:
workervec = [("jovyan@10.0.0.29:3456", 2),("jovyan@10.0.75.1:3456", 1)]
addprocs(workervec ; sshflags=`-i /home/jovyan/.ssh/pubkey`, 
         tunnel=true, exename=`/usr/local/bin/julia`)



3-element Array{Int64,1}:
 2
 3
 4

In [3]:
@everywhere using Distributed

In [5]:
@everywhere using PyCall

In [6]:
@everywhere @pyimport socket

In [7]:
@everywhere function printhello(data)
    println("hi from node ",myid() )
    print("my hostnameis ", socket.gethostname())
    println(" and the data is ", data)
end

In [8]:
workers()

3-element Array{Int64,1}:
 2
 3
 4

In [9]:
x = [@spawnat i printhello( i*10 ) for i in workers()]

3-element Array{Future,1}:
 Future(2, 1, 42, nothing)
 Future(3, 1, 43, nothing)
 Future(4, 1, 44, nothing)

      From worker 3:	hi from node 3
      From worker 3:	my hostnameis 4f6329fc6694 and the data is 30
      From worker 4:	hi from node 4
      From worker 4:	my hostnameis 4f6329fc6694 and the data is 40
      From worker 2:	hi from node 2
      From worker 2:	my hostnameis bdfb4a7068e2 and the data is 20


In [ ]:
fetch(x)

In [10]:
ch = RemoteChannel(()->Channel{String}(1000), 1)

RemoteChannel{Channel{String}}(1, 1, 45)

In [11]:
@everywhere function listen(ch)
    println("listen started")
    row = 0
    id = "1"
    while id != "-1"
        id = take!(ch)
        x = string("out from ",string(myid())," val=", string(id), " row =", string(row))
        println(x)
        #putInTable(tbl,row, x )
        row = row+1
    end
    println("listen done")
end

In [12]:
x = [@spawnat i listen(ch) for i in workers()]

3-element Array{Future,1}:
 Future(2, 1, 52, nothing)
 Future(3, 1, 53, nothing)
 Future(4, 1, 54, nothing)

      From worker 3:	listen started
      From worker 4:	listen started
      From worker 2:	listen started


In [13]:
function putInCh(ch)
    for i=1:4
        put!(ch, string(i))
        println(i)
    end
end

putInCh (generic function with 1 method)

In [14]:
putInCh(ch)

1
2
3
4
      From worker 3:	out from 3 val=2 row =0
      From worker 4:	out from 4 val=1 row =0
      From worker 2:	out from 2 val=3 row =0
      From worker 3:	out from 3 val=4 row =1


In [15]:
put!(ch, string(-1))

RemoteChannel{Channel{String}}(1, 1, 45)

      From worker 4:	out from 4 val=-1 row =1
      From worker 4:	listen done


In [16]:
put!(ch, string(-1))

RemoteChannel{Channel{String}}(1, 1, 45)

      From worker 2:	out from 2 val=-1 row =1
      From worker 2:	listen done


In [17]:
put!(ch, string(-1))

RemoteChannel{Channel{String}}(1, 1, 45)

      From worker 3:	out from 3 val=-1 row =2
      From worker 3:	listen done
